# next()和send()

实际上next()和send()在一定意义上作用是相似的，区别是send()可以传递yield表达式的值进去，而next()不能传递特定的值，只能传递None进去。因此，我们可以看做calc_average.next() 和 calc_average.send(None) 作用是一样的。

为什么第一次调用时，需使用next()语句或是send(None)
PEP 342中有
> 
Because generator-iterators begin execution at the top of the
    generator's function body, there is no yield expression to receive
    a value when the generator has just been created.  Therefore,
    calling send() with a non-None argument is prohibited when the
    generator iterator has just started, and a TypeError is raised if
    this occurs (presumably due to a logic error of some kind).  Thus,
    before you can communicate with a coroutine you must first call
    next() or send(None) to advance its execution to the first yield
    expression.

https://www.zhihu.com/question/28105502


下面说明下send执行的顺序。new_num = yield average这句话是从右往左执行的。当第一次send（None）（对应27行）时，启动生成器，从生成器函数的第一行代码开始执行，直到第一次执行完yield（对应第7行）后，**跳出生成器函数**。这个过程中，new_num一直没有定义。

运行到send（10）时，进入生成器函数，此时，将yield average看做一个整体，赋值给它并且传回。此时即相当于把10赋值给new_num，但是并不执行yield部分。下面继续从yield的下一语句继续执行，然后重新运行到yield语句，执行后，跳出生成器函数。即send和next相比，只是开始多了一次赋值的动作，其他运行流程是相同的。


In [112]:
# 子生成器
def average_gen():
    total = 0
    count = 0
    average = 0
    while True:
        new_num = yield average
        if new_num is None:
            break
        count += 1
        total += new_num
        average = total/count

    # 每一次return，都意味着当前协程结束。
    return total,count,average

# 委托生成器
def proxy_gen():
    while True:
        # 只有子生成器要结束（return）了，yield from左边的变量才会被赋值，后面的代码才会执行。
        total, count, average = yield from average_gen()
        print("计算完毕！！\n总共传入 {} 个数值， 总和：{}，平均数：{}".format(count, total, average))

# 调用方
def main():
    calc_average = proxy_gen()
    next(calc_average)            # 预激协程
    print(calc_average.send(10))  # 打印：10.0
    print(calc_average.send(20))  # 打印：15.0
    print(calc_average.send(30))  # 打印：20.0
    calc_average.send(None)      # 结束协程
    # 如果此处再调用calc_average.send(10)，由于上一协程已经结束，将重开一协程


main()



10.0
15.0
20.0
计算完毕！！
总共传入 3 个数值， 总和：60，平均数：20.0
